inisialisasi ke database

In [1]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("/content/latihanujikom-85ee2-firebase-adminsdk-400lb-7d5f95374f.json")
firebase_admin.initialize_app(cred)

mengambil collection dan document di dalamnya

In [2]:
from firebase_admin import firestore

db = firestore.client() #inisialisasi db sebagai database

# data_db = db.collection('ujikom') # mengambil collecction data
# ujikom_ref = db.collection('ujikom')

# data_doc = data_db.document('data5') # ngambil id doc nya

barang_ref = db.collection('barang')  # Collection untuk data barang
transaksi_ref = db.collection('transaksi')  # Collection untuk data transaksi

barang = barang_ref.document('test') # ngambil id doc nya
transaksi = transaksi_ref.document('test')

def testConnecttion(barang,transaksi):

  barang.set(
      {
          'test':'hello dek'
      }
  )

  transaksi.set(
      {
          'test':'hello dek'
      }
  )

  print(barang.get().to_dict()) #unit test connection database
  print(transaksi.get().to_dict()) #unit test connection database

testConnecttion(barang,transaksi)

{'test': 'hello dek'}
{'test': 'hello dek'}


reset database data.data5 jadi kosong

In [3]:
#function ini untuk menghapus data yang dihasilkan dari uji connection
def resetDatabase(barang,transaksi):
  barang.delete()
  print(barang.get().to_dict())
  print("database sudah kosong")

  transaksi.delete()
  print(transaksi.get().to_dict())
  print("database sudah kosong")

resetDatabase(barang,transaksi)

None
database sudah kosong
None
database sudah kosong


Data process

In [4]:

import pandas as pd

# 4. Fungsi untuk Menambahkan Data Barang
def tambah_barang(nama, ukuran, harga, stok):
  data_barang = {
      'nama': nama,
      'ukuran': ukuran,
      'harga': harga,
      'stok': stok
  }
  barang_ref.document(nama).set(data_barang)  # Menyimpan di collection 'barang'
  print("Data barang berhasil ditambahkan!")

# 5. Fungsi untuk Menambahkan Data Transaksi
def tambah_transaksi(nama_pelanggan, tanggal, nama_beras, ukuran, jumlah, keterangan=""):
    doc_ref = barang_ref.document(nama_beras)  # Mengambil data barang dari collection 'barang'
    doc = doc_ref.get()

    if doc.exists:
        data_barang = doc.to_dict()
        if data_barang['stok'] >= jumlah:
            data_barang['stok'] -= jumlah
            doc_ref.set(data_barang)  # Memperbarui stok di collection 'barang'

            # Menghitung total harga dengan pajak
            harga_beras = data_barang['harga']
            total_harga = harga_beras * jumlah
            pajak = 0.11 * total_harga
            total_harga_pajak = total_harga + pajak

            data_transaksi = {
                'nama_pelanggan': nama_pelanggan,
                'tanggal': tanggal,
                'nama_beras': nama_beras,
                'ukuran': ukuran,
                'jumlah': jumlah,
                'keterangan': keterangan,
                'total_harga': total_harga_pajak,
                'keterangan_pajak': f"(+ Pajak 11% = Rp {pajak:.2f})"
            }
            transaksi_ref.add(data_transaksi)  # Menyimpan di collection 'transaksi'
            print("Transaksi berhasil!")
        else:
            print("Stok tidak mencukupi!")
    else:
        print("Barang tidak ditemukan!")

# 6. Fungsi untuk Menampilkan Daftar Barang
def tampilkan_barang():
  docs = barang_ref.stream()  # Mengambil data dari collection 'barang'
  data_barang = []
  for doc in docs:
    data_barang.append(doc.to_dict())
  df = pd.DataFrame(data_barang)
  print(df.to_markdown(index=False, numalign='left', stralign='left'))

# 7. Fungsi untuk Menampilkan Daftar Transaksi
def tampilkan_transaksi():
    docs = transaksi_ref.stream()
    data_transaksi = []
    for doc in docs:
        data_transaksi.append(doc.to_dict())
    df = pd.DataFrame(data_transaksi)

    # Pastikan kolom 'total_harga' dan 'keterangan_pajak' ada sebelum menampilkan
    if 'total_harga' not in df.columns or 'keterangan_pajak' not in df.columns:
        print("Data transaksi belum lengkap.")
        return

    print(df[['nama_pelanggan', 'tanggal', 'nama_beras', 'ukuran', 'jumlah', 'keterangan', 'total_harga', 'keterangan_pajak']].to_markdown(index=False, numalign='left', stralign='left'))

# 8. Fungsi untuk Mencari Barang dan Transaksi
def cari_barang_dan_transaksi(kata_kunci):
  # Cari barang yang mengandung kata kunci (case-insensitive)
  query_barang = barang_ref.where("nama", ">=", kata_kunci.lower()).where("nama", "<=", kata_kunci.upper() + "\uf8ff")
  docs_barang = query_barang.stream()
  data_barang = []
  for doc in docs_barang:
    data_barang.append(doc.to_dict())

  # Cari transaksi yang mengandung kata kunci (case-insensitive)
  query_transaksi = transaksi_ref.where("nama_beras", ">=", kata_kunci.lower()).where("nama_beras", "<=", kata_kunci.upper() + "\uf8ff")
  docs_transaksi = query_transaksi.stream()
  data_transaksi = []
  for doc in docs_transaksi:
    data_transaksi.append(doc.to_dict())

  # Tampilkan hasil pencarian dalam bentuk tabel
  if data_barang:
    df_barang = pd.DataFrame(data_barang)
    print("\n--- Hasil Pencarian Barang ---")
    print(df_barang.to_markdown(index=False, numalign='left', stralign='left'))
  else:
    print("Tidak ada barang yang ditemukan.")

  if data_transaksi:
    df_transaksi = pd.DataFrame(data_transaksi)
    print("\n--- Hasil Pencarian Transaksi ---")
    print(df_transaksi.to_markdown(index=False, numalign='left', stralign='left'))
  else:
    print("Tidak ada transaksi yang ditemukan.")


Main program


In [5]:
5# 8. Main Program
def main():
  while True:
    print("\n--- Menu ---")
    print("1. Tambah data barang")
    print("2. Tambah data transaksi")
    print("3. Tampilkan data barang")
    print("4. Tampilkan data transaksi")
    print("5. Cari barang dan transaksi") # Menambahkan pilihan baru
    print("6. Keluar") # Mengubah pilihan keluar menjadi 6

    pilihan = input("Masukkan pilihan: ")

    if pilihan == '1':
      nama = input("Nama beras: ")
      ukuran = input("Ukuran (kg): ")
      harga = int(input("Harga: "))
      stok = int(input("Stok: "))
      tambah_barang(nama, ukuran, harga, stok)
    elif pilihan == '2':
      nama_pelanggan = input("Nama pelanggan: ")
      tanggal = input("Tanggal (YYYY-MM-DD): ")
      nama_beras = input("Nama beras: ")
      ukuran = input("Ukuran (kg): ")
      jumlah = int(input("Jumlah: "))
      tambah_transaksi(nama_pelanggan, tanggal, nama_beras, ukuran, jumlah)
    elif pilihan == '3':
      tampilkan_barang()
    elif pilihan == '4':
      tampilkan_transaksi()
    elif pilihan == '5': # Pilihan untuk fitur pencarian
      kata_kunci = input("Masukkan kata kunci: ")
      cari_barang_dan_transaksi(kata_kunci)
    elif pilihan == '6':  # Pilihan untuk keluar
      break
    else:
      print("Pilihan tidak valid!")


RUN BAGIAN INI TERAKHIR SETELAH RUN BAGIAN BAGIAN LAIN



FUNGSI INI BISA DIJALANKAN BERULANG ULANG DALAM SARAT MASIH 1 RUNTIME

In [6]:
main()


--- Menu ---
1. Tambah data barang
2. Tambah data transaksi
3. Tampilkan data barang
4. Tampilkan data transaksi
5. Cari barang dan transaksi
6. Keluar
Masukkan pilihan: 5
Masukkan kata kunci: Zaytun


/usr/local/lib/python3.10/dist-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
<ipython-input-4-2b8403925ad5>:75: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  query_barang = barang_ref.where("nama", ">=", kata_kunci.lower()).where("nama", "<=", kata_kunci.upper() + "\uf8ff")
<ipython-input-4-2b8403925ad5>:82: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  query_transaksi = transaksi_ref.where("nama_beras", ">=", kata_kunci.lower()).where("nama_beras", "<=", kata_kunci.upper() + "\uf8ff")


Tidak ada barang yang ditemukan.
Tidak ada transaksi yang ditemukan.

--- Menu ---
1. Tambah data barang
2. Tambah data transaksi
3. Tampilkan data barang
4. Tampilkan data transaksi
5. Cari barang dan transaksi
6. Keluar
Masukkan pilihan: 3
| stok   | nama                  | ukuran   | harga   |
|:-------|:----------------------|:---------|:--------|
| 89     | BMW Setra Ramos       | 5        | 68000   |
| 32     | Bunga Ramos Setra     | 10       | 120000  |
| 99     | Maknyuss Premium      | 25       | 338000  |
| 37     | Pandan Zaytun         | 5        | 90000   |
| 6      | Puregreen Beras Merah | 2        | 45000   |
| 7      | Rojolele Super        | 25       | 300000  |
| 118    | Topi Koki Setra Ramos | 5        | 73000   |

--- Menu ---
1. Tambah data barang
2. Tambah data transaksi
3. Tampilkan data barang
4. Tampilkan data transaksi
5. Cari barang dan transaksi
6. Keluar
Masukkan pilihan: 4
| nama_pelanggan     | tanggal    | nama_beras            | ukuran   | jumlah   |